In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal, traceback
import numpy as np
import copy

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x, debug=False):
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    if debug:
                        print(x)
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
        if debug:
            return x
        else:
            exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)
    

In [2]:
###
### Display ###
###

_IMPORT('import pandas as pd')
_IMPORT('import cv2')
_IMPORT('from PIL import Image')
_IMPORT('import matplotlib.pyplot as plt')
_IMPORT('import plotly')
_IMPORT('import plotly.graph_objects as go')
_IMPORT('import ipywidgets as widgets')
_IMPORT('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)
%matplotlib notebook

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<center><img %s %s src="%s"/></center>' % (W, H, data_url))

def nbeasy_widget_display(images, img_wid=None):
    if isinstance(images, np.ndarray):
        images = {'_': images}
    elif isinstance(images, tuple) or isinstance(images, list):
        images = {f'_{i}': img for i, img in enumerate(images)}
    C = len(images)
    if C == 0:
        return None
    show_ncol, show_nrow = 1, 1
    if C > 1:
        if img_wid:
            show_ncol = 2 if int(img_wid.width) < 1280 else 1
        for i in range(C % show_ncol):
            images[f'placehold-{i}'] = images[list(images.keys())[-1]].copy()
        show_nrow = len(images) // show_ncol
        row_images = []
        col_images = []
        for key, img in images.items():
            if not key.startswith('_'):
                cv2.putText(img, key, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 9,2), 1)
            col_images.append(img)
            if len(col_images) == show_ncol:
                row_images.append(np.hstack(col_images))
                col_images = []

        display_image = np.vstack(row_images)
    else:
        display_image = images.popitem()[1]
    
    if img_wid:
        img_wid.layout.width = f'{display_image.shape[1]}px'
        img_wid.layout.height = f'{display_image.shape[0]}px'
        if isinstance(img_wid, widgets.Image):
            img_wid.value = io.BytesIO(cv2.imencode('.png', display_image)[1]).getvalue()
        else:
            img_wid.height, img_wid.width = display_image.shape[0], display_image.shape[1]
            img_wid.put_image_data(cv2.cvtColor(display_image, cv2.COLOR_BGR2RGB))
    else:
        return imgbytes

# https://www.webucator.com/article/python-color-constants-module/
class COLORS(object):
    # BGR
    BLUE       = (255 , 0   , 0)
    GREEN      = (0   , 255 , 0)
    RED        = (0   , 0   , 255)
    BLACK      = (0   , 0   , 0)
    YELLOW     = (0   , 255 , 255)
    WHITE      = (255 , 255 , 255)
    CYAN       = (255 , 255 , 0)
    MAGENTA    = (255 , 0   , 242)
    GOLDEN     = (32  , 218 , 165)
    LIGHT_BLUE  = (255 , 9   , 2)
    PURPLE     = (128 , 0   , 128)
    CHOCOLATE  = (30  , 105 , 210)
    PINK       = (147 , 20  , 255)
    ORANGE     = (0   , 69  , 255)
    GRAY       = (125 , 125 , 125)
    DARKGRAY   = (50  , 50  , 50)
    DARKGREEN  = (0   , 100 , 0)
    OLIVE      = (0   , 128 , 0)
    SILVER     = (192 , 192 , 192)

In [3]:
import json, functools, datetime

class __JsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.datetime, datetime.timedelta)):
            return '{}'.format(obj)
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return json.JSONEncoder.default(self, obj)

json.dumps = functools.partial(json.dumps, cls=__JsonEncoder)

In [4]:
import skimage.filters as filters
import threading
import shutil
import glob
import pickle
import collections
from enum import IntEnum

_IMPORT('gitee.com/qrsforever/nb_easy/easy_widget')

g_ctx = None

In [5]:
_IMPORT('./easy_widget.py')

## Binocular: Stereo Camera Calibration

[理论参考](https://string.quest/read/6034570)

### Utils 

In [6]:
def _get_cameras():
    prefix = '/sys/class/video4linux/'
    cameras = [('None', 'none')]
    if os.path.isdir(prefix):
        for dev in os.listdir(prefix):
            with open(f'{prefix}/{dev}/name', 'r') as f:
                name = f.readline().strip().split(':')[0]
                ty = 'monocular'
                if '3D' in name:
                    ty = 'binocular'
                cameras.append((name, (f'/dev/{dev}', ty)))
    return cameras
        
    
VIDEO_MAX_SIZE = 10000

class StereoCamera(object):
    def __init__(self, source, resolution, is_3d = False):
        self.video_source = source
        self.video_capture = None
        self.is_running = False
        self.width, self.height = resolution
        self._is_3d = is_3d
        # self.grabbed, self.frame = False, None
        # self.read_lock, self.read_thread = threading.Lock(), None
        
    def is_3d(self):
        return self._is_3d
        
    # def start(self):
    #     if self.is_running:
    #         return
    #     self.grabbed, self.frame = self.video_capture.read()
    #     if self.grabbed:
    #         self.is_running = True
    #         self.read_thread = threading.Thread(target=self._update_frame)
    #         self.read_thread.start()
    #         
    # def stop(self):
    #     if self.is_running:
    #         self.is_running = False 
    #         if self.video_capture:
    #             self.video_capture.release()
    #         if self.read_thread:
    #             self.read_thread.join()
    #     self.video_capture, self.read_thread = None, None
    #     
    # def _update_frame(self):
    #     try:
    #         while self.is_running:
    #             grabbed, frame = self.video_capture.read()
    #             with self.read_lock:
    #                 self.grabbed = grabbed
    #                 self.frame = frame
    #     finally:
    #         self.stop()
    #         
    # def read(self):
    #     with self.read_lock:
    #         grabbed = self.grabbed
    #         frame = self.frame.copy()
    #     return grabbed, frame
    # 
    # def read_left_right(self):
    #     with self.read_lock:
    #         frame = self.frame.copy()
    #         grabbed = self.grabbed
    #         frameL = frame[:, :int(self.width / 2), :]
    #         frameR = frame[:, int(self.width / 2):, :]
    #     return grabbed, frameL, frameL
    
    def open(self):
        video_capture = cv2.VideoCapture(self.video_source)
        if not video_capture.isOpened():
            raise RuntimeError(f'Cannot open {self.video_source}')
            
        if self._is_3d:
            video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, VIDEO_MAX_SIZE) # 2560
            video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, VIDEO_MAX_SIZE) # 960
            width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            if width != 2560:
                video_capture.release()
                raise RuntimeError(f'{self.video_source} is not stereo camera!')
            video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, self.width * 2)
        else:
            video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, self.width)
        video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, self.height)
            
        self.width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        
        self.video_capture = video_capture
        self.is_running = True
        return self.video_capture.read()[1].shape
        
    def close(self):
        self.is_running = False
        for _ in range(10):
            if self.video_capture is None:
                break
            time.sleep(0.3)
    
    def read(self, flip=None):
        try:
            idx = 0
            while self.is_running:
                grabbed, frame = self.video_capture.read()
                if not grabbed:
                    raise RuntimeError('Video read error!')
                idx += 1
                
                if flip > -2:
                    frame = cv2.flip(frame, flip)
                if self._is_3d:
                    # 分离左右摄像头
                    frameL = frame[:, :int(self.width / 2), :]
                    frameR = frame[:, int(self.width / 2):, :]
                    yield idx, [frameL, frameR]
                else:
                    yield idx, [frame]
        finally:
            self.video_capture.release()
            self.video_capture = None
            
        raise StopIteration
        

def start_calibrate_camera_with_good_flags(ctx, calibrate, img_size, thresh_rate=1.25):
    flags = 0
    K = np.array([[max(img_size), 0, img_size[1]/2],[0, max(img_size), img_size[0]/2], [0, 0, 1]])
    D = np.zeros((5, 1))
    
    # An RMS error of 1.0 means that, on average, each of these projected points is 1.0 px away from its actual position. 
    # [0.0, 1.0] is good
    retval, mat, dist, rvecs, tvecs, \
    std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)

    ctx.logger(json.dumps({
        're-project-error': retval, 
        'mat': str(mat.round(3).tolist()),
        'dist': str(dist.round(3).tolist())}, indent=4))

    aspect_ratio = mat[0][0] / mat[1][1]
    if 1.0 - min(aspect_ratio, 1.0/aspect_ratio) < 0.01:
        flags += cv2.CALIB_FIX_ASPECT_RATIO
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'aspect_ratio': aspect_ratio,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    center_point_reldiff = max(abs(np.array(mat[0, 2], mat[1][2]) - np.array(img_size)/2) / np.array(img_size))
    if center_point_reldiff < 0.05:
        flags += cv2.CALIB_FIX_PRINCIPAL_POINT
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'center_point_reldiff': center_point_reldiff,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    error_threshold = thresh_rate * retval
    camera_matrix = mat
    dist_coeffs = dist
    error = retval

    ignore_flags = {
        'ignore_tangential_distortion': cv2.CALIB_ZERO_TANGENT_DIST,
        'ignore_k3': cv2.CALIB_FIX_K3,
        'ignore_k2': cv2.CALIB_FIX_K2,
        'ignore_k1': cv2.CALIB_FIX_K1
    }

    for k, v in ignore_flags.items():
        flags += v
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'ignore_type': k,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))
        if retval > error_threshold:
            continue
        camera_matrix = mat
        dist_coeffs = dist
        error = retval
    
    return error, camera_matrix, dist_coeffs

            
def detect_chessboard(image, pattern_size, criteria, win_size, draw=False):
    # 检测棋盘格
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    better_corners = None
    found, corners = cv2.findChessboardCorners(gray, pattern_size, flags=cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
    if found:
        # 角点精检测 (亚像素)
        better_corners = cv2.cornerSubPix(gray, corners, win_size, (-1, -1), criteria)
        if draw:
            cv2.drawChessboardCorners(image, pattern_size, better_corners, found)
    return better_corners


def calibrate_monocular_camera(ctx, imgfiles, world_point, pattern_size, criteria, win_size):        
    image_size = None
    world_points, pixel_points = [], []
    for ifile in imgfiles:
        image = cv2.imread(ifile)
        if image_size is None:
            image_size = image.shape[::-1][1:]
        pixel_point = detect_chessboard(image, pattern_size, criteria, win_size)
        if pixel_point is not None: 
            world_points.append(world_point)
            pixel_points.append(pixel_point)
 
    error, mtx, dist = start_calibrate_camera_with_good_flags(ctx,
        functools.partial(cv2.calibrateCameraExtended, objectPoints=world_points, imagePoints=pixel_points),
        img_size=image_size)
    
    return image_size, error, mtx, dist


def calibrate_binocular_camera(ctx, img_l_files, img_r_files, world_point, pattern_size, criteria, win_size, flags):
    ctx.logger(f'calibrate_binocular_camera({flags})')
    image_size = None
    world_points, pixel_points_L, pixel_points_R = [], [], []
    for lfile, rfile in zip(img_l_files, img_r_files):
        image_L = cv2.imread(lfile)
        image_R = cv2.imread(rfile)
        if image_size is None:
            image_size = image_L.shape[::-1][1:]
            ctx.logger(f'{lfile}, {rfile}, {image_size}')
        pixel_point_L = detect_chessboard(image_L, pattern_size, criteria, win_size)
        pixel_point_R = detect_chessboard(image_R, pattern_size, criteria, win_size)
        if pixel_point_L is not None and pixel_point_R is not None:
            world_points.append(world_point)
            pixel_points_L.append(pixel_point_L)
            pixel_points_R.append(pixel_point_R)
 
    #  error_L, mtx_L, dist_L = start_calibrate_camera_with_good_flags(ctx,
    #      functools.partial(cv2.calibrateCameraExtended, objectPoints=world_points, imagePoints=pixel_points_L),
    #      img_size=image_size, thresh_rate=1.02)
    #  
    #  error_R, mtx_R, dist_R = start_calibrate_camera_with_good_flags(ctx,
    #      functools.partial(cv2.calibrateCameraExtended, objectPoints=world_points, imagePoints=pixel_points_R),
    #      img_size=image_size, thresh_rate=1.02)
    
    # expect: error less than 1.0
    error_L, mtx_L, dist_L, _, _ = cv2.calibrateCamera(world_points, pixel_points_L, image_size, None, None)
    error_R, mtx_R, dist_R, _, _ = cv2.calibrateCamera(world_points, pixel_points_R, image_size, None, None)
    
    ctx.logger(f'L: {error_L}\n {mtx_L}\n {dist_L}')
    ctx.logger(f'R: {error_R}\n {mtx_R}\n {dist_R}')

    # flags = CALIB_FIX_INTRINSIC # default
    # flags = 0
    # flags |= cv2.CALIB_FIX_INTRINSIC # fix the intrinsic camara matrixes so that only R, T, E and F are calculated
    # flags |= cv2.CALIB_USE_INTRINSIC_GUESS # initial value of cameraMatrix and distCoeffs provided by the user
    # flags |= cv2.CALIB_FIX_FOCAL_LENGTH # the focal length is not changed during the iteration
    # flags |= cv2.CALIB_ZERO_TANGENT_DIST # the tangential distortion remains zero
    error, K1, D1, K2, D2, R, T, E, F = cv2.stereoCalibrate(
        world_points, pixel_points_L, pixel_points_R,
        mtx_L, dist_L, mtx_R, dist_R, image_size,
        R=None, # 第一和第二个摄像机之间的旋转矩阵
        T=None, # 第一和第二个摄像机之间的平移矩阵
        E=None, # essential matrix本质矩阵
        F=None, # fundamental matrix基本矩阵
        flags=flags,
        criteria=criteria
    )
    
    # D1 = np.zeros((5, 1))
    # D2 = np.zeros((5, 1))
    
    ctx.logger(f'error_L: {error_L} error_R: {error_R} error: {error}')

    return image_size, error, K1, D1, K2, D2, R, T, E, F


def rectify_binocular_camera(ctx, K1, D1, K2, D2, image_size, R, T, alpha):
    """
    alpha  -1: 自动剪切 0: 没有黑边  1: 所有像素(有黑边)
    """
    R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(
        K1, D1, K2, D2, image_size, R, T,
        R1=None, # 第一个摄像机的校正变换矩阵（旋转变换）
        R2=None, # 第二个摄像机的校正变换矩阵（旋转变换）
        P1=None, # 第一个摄像机在**新**坐标系下的投影矩阵
        P2=None, # 第二个摄像机在**新**坐标系下的投影矩阵
        Q=None,  # 4*4的视差图到深度图的映射矩阵(disparity-to-depth mapping matrix )
        flags=cv2.CALIB_ZERO_DISPARITY, alpha=alpha, newImageSize=None)
    
    # 畸变校正和立体校正的映射矩阵, 计算像素空间坐标的重投影矩阵
    l_map_x, l_map_y = cv2.initUndistortRectifyMap(K1, D1, R1, P1, image_size, cv2.CV_32FC1)
    r_map_x, r_map_y = cv2.initUndistortRectifyMap(K2, D2, R2, P2, image_size, cv2.CV_32FC1)
    return l_map_x, l_map_y, r_map_x, r_map_y, R1, R2, P1, P2, Q


def get_depth(disparity, Q, scale=1.0, method=False):
    # Q: 重投影矩阵
    # Q = [
    #    [1,   0,       0,         -cx]
    #    [0,   1,       0,         -cy]
    #    [0,   0,       0,           f]
    #    [1,   0,   -1/Tx, (cx-cx`)/Tx]
    # ]
    
    if method:
        points_3d = cv2.reprojectImageTo3D(disparity, Q)  # 单位是毫米(mm)
        x, y, depth = cv2.split(points_3d)
    else:
        baseline = 1 / Q[3, 2]
        fx = abs(Q[2, 3])
        depth = (fx * baseline) / disparity

    return np.asarray(depth * scale, dtype=np.float32)

### Callback Events

#### On Click Events

In [7]:
# folding

def on_start_collect_samples(ctx, w_btn, camera_source, camera_resolution, sample_size, flip, rm_out, save_dir, win_size, term_iters, term_eps, w_video):
    segs = os.path.basename(save_dir).split('_')
    # squares_x, squares_y, square_size, win_size, term_iters, term_eps = *list(map(int, segs[:-1])), float(segs[-1])
    squares_x, squares_y, square_size = list(map(int, segs))
    ctx.logger('on_start_collect_samples(%s, %s, %d, %d, %d, %d, %d, %d, %f, %d)' % (
        ':'.join(camera_source), camera_resolution, sample_size, squares_x, squares_y, square_size, win_size, term_iters, term_eps,
    flip), clear=1)
    
    pattern_size = (squares_x - 1, squares_y - 1)
    win_size = (win_size, win_size)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)
    
    if rm_out:
        shutil.rmtree(save_dir, ignore_errors=True)
    os.makedirs(save_dir, exist_ok=True)
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            count = 0
            for idx, frames in camera.read(flip=flip):
                if idx % camera.fps == 0:
                    if len(frames) == 2:
                        frameL_copied, frameR_copied = frames[0].copy(), frames[1].copy()
                        cornersL = detect_chessboard(frameL_copied, pattern_size, criteria, win_size, draw=True)
                        cornersR = detect_chessboard(frameR_copied, pattern_size, criteria, win_size, draw=True)
                        if cornersL is not None and cornersR is not None:
                            count += 1
                            cv2.imwrite(os.path.join(save_dir, "left_ori_{:0=3d}.png".format(count)), frames[0])
                            cv2.imwrite(os.path.join(save_dir, "left_out_{:0=3d}.png".format(count)), frameL_copied)

                            cv2.imwrite(os.path.join(save_dir, "right_ori_{:0=3d}.png".format(count)), frames[1])
                            cv2.imwrite(os.path.join(save_dir, "right_out_{:0=3d}.png".format(count)), frameR_copied)

                            cv2.putText(frameL_copied, f'Count: {count}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                            nbeasy_widget_display({'L': frameL_copied, 'R': frameR_copied}, w_video, resize="auto")

                            if count == sample_size:
                                break
                        else:
                            nbeasy_widget_display({'L': frameL_copied, 'R': frameR_copied}, w_video, resize="auto")
                            
                    else:
                        frame_copied = frames[0]
                        corners = detect_chessboard(frame_copied, pattern_size, criteria, win_size, draw=True)
                        if corners is not None:
                            count += 1
                            cv2.imwrite(os.path.join(save_dir, "ori_{:0=3d}.png".format(count)), frames[0])
                            cv2.imwrite(os.path.join(save_dir, "out_{:0=3d}.png".format(count)), frame_copied)
                            cv2.putText(frame_copied, f'Count: {count}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                            nbeasy_widget_display(frame_copied, w_video)

                            if count == sample_size:
                                break
                        else:
                            nbeasy_widget_display(frameL_copied, w_video)
        except (StopIteration, RuntimeError):
            pass
        finally:
            camera.close()
            w_btn.disabled = False

    w_video.width, w_video.height = camera_resolution
    camera = StereoCamera(camera_source[0], camera_resolution, camera_source[1] == 'binocular')
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()


def on_stop_collect_samples(ctx, w_btn = None):
    ctx.logger("on_stop_collect_samples")
    if hasattr(ctx, 'stereo_camera'):
        ctx.stereo_camera.close() 

        
def on_start_calibrition_test(ctx, w_btn, camera_source, camera_resolution, flip, save_dir, calibration_result, w_video):
    ctx.logger('on_start_calibrition_test(%s, %s, %s)' % (':'.join(camera_source), camera_resolution, save_dir), clear=1)
    is_3d = camera_source[1] == 'binocular'
    if not is_3d:
        ctx.logger('not 3d camera')
        return
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            params = json.loads(calibration_result)

            K1, D1 = np.asarray(params['K1']), np.asarray(params['D1'])
            K2, D2 = np.asarray(params['K2']), np.asarray(params['D2'])
            mapxL, mapyL, mapxR, mapyR = None, None, None, None
            newcameramtxL, roiL, newcameramtxR, roiR = None, None, None, None
            alpha = 0
            for idx, frames in camera.read(flip=flip):
                display_frames = {}
                if len(frames) == 2:
                    imgL, imgR = frames
                    display_frames['L'] = imgL
                    display_frames['R'] = imgR
                    t1 = time.time() * 1000
                    undistort_imgL = cv2.undistort(imgL.copy(), K1, D1)
                    t2 = time.time() * 1000
                    undistort_imgR = cv2.undistort(imgR.copy(), K2, D2)
                    t3 = time.time() * 1000
                    display_frames[f'undist L {int(t2 - t1)}'] = undistort_imgL 
                    display_frames[f'undist R {int(t3 - t2)}'] = undistort_imgR
                    
                    # fast than undistort (speedup 2x - 5x)
                    if mapxL is None:
                        h, w = imgL.shape[:2]
                        newcameramtxL, roiL = cv2.getOptimalNewCameraMatrix(K1, D1, (w, h), alpha=alpha, newImgSize=(w, h))
                        newcameramtxR, roiR = cv2.getOptimalNewCameraMatrix(K2, D2, (w, h), alpha=alpha, newImgSize=(w, h))
                        mapxL, mapyL = cv2.initUndistortRectifyMap(K1, D1, None, newcameramtxL, (w, h), cv2.CV_32FC1)
                        mapxR, mapyR = cv2.initUndistortRectifyMap(K2, D2, None, newcameramtxR, (w, h), cv2.CV_32FC1)                        
                    t4 = time.time() * 1000
                    remap_imgL = cv2.remap(imgL.copy(), mapxL, mapyL, cv2.INTER_LINEAR)
                    if alpha != 0:
                        remap_imgL = remap_imgL[roiL[1]: roiL[1] + roiL[3], roiL[0]: roiL[0] + roiL[2]]
                    t5 = time.time() * 1000
                    remap_imgR = cv2.remap(imgR.copy(), mapxR, mapyR, cv2.INTER_LINEAR)
                    if alpha != 0:
                        remap_imgR = remap_imgR[roiL[1]: roiR[1] + roiR[3], roiR[0]: roiR[0] + roiR[2]]
                    t6 = time.time() * 1000
                    display_frames[f'remap L {int(t5 - t4)}'] = cv2.resize(remap_imgL, (w, h))
                    display_frames[f'remap R {int(t6 - t5)}'] = cv2.resize(remap_imgR, (w, h))
                    nbeasy_widget_display(display_frames, w_video, resize="auto", ctx=ctx)
                else:
                    raise NotImplemented
        except (StopIteration, RuntimeError):
            pass
        except Exception:
            ctx.logger(f'{traceback.format_exc(limit=3)}')
        finally:
            camera.close()
            w_btn.disabled = False

    w_video.width, w_video.height = camera_resolution
    camera = StereoCamera(camera_source[0], camera_resolution, is_3d)
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()
    
    
on_stop_test = on_stop_collect_samples


def on_start_rectification_test(ctx, w_btn, camera_source, camera_resolution, flip, save_dir, rectification_result, w_video):
    ctx.logger('on_start_rectification_test(%s, %s)' % (':'.join(camera_source), save_dir), clear=1)
    is_3d = camera_source[1] == 'binocular'
    if not is_3d:
        ctx.logger('not 3d camera')
        return
    
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            params = json.loads(rectification_result)

            K1, D1 = np.asarray(params['K1']), np.asarray(params['D1'])
            K2, D2 = np.asarray(params['K2']), np.asarray(params['D2'])
            
            l_map, r_map = np.load(params['l_map_npz']), np.load(params['r_map_npz'])
            l_map_x, l_map_y = l_map['l_map_x'], l_map['l_map_y']
            r_map_x, r_map_y = r_map['r_map_x'], r_map['r_map_y']
            interval = 25
            for idx, frames in camera.read(flip=flip):
                display_frames = {}
                if len(frames) == 2:
                    imgL, imgR = frames
                    display_frames['undist L'] = cv2.undistort(imgL.copy(), K1, D1)
                    display_frames['undist R'] = cv2.undistort(imgR.copy(), K2, D2)
                    
                    rectifiedL = cv2.remap(imgL, l_map_x, l_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    rectifiedR = cv2.remap(imgR, r_map_x, r_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    
                    # H = max(rectifiedL.shape[0], rectifiedR.shape[0])
                    # W = rectifiedL.shape[1] + rectifiedR.shape[1]
                    # rectify_image = np.zeros((H, W, 3), dtype=np.uint8)
                    # rectify_image[0:rectifiedL.shape[0], 0:rectifiedL.shape[1]] = rectifiedL
                    # rectify_image[0:rectifiedR.shape[0],  rectifiedL.shape[1]:] = rectifiedR
                    # # 绘制等间距平行线
                    # for k in range(H // interval):
                    #     cv2.line(rectify_image, (0, interval * (k + 1)), (2 * W, interval * (k + 1)), COLORS.RED, 1, lineType=cv2.LINE_AA)
                    for k in range(rectifiedL.shape[0] // interval):
                        y = interval * (k + 1)
                        cv2.line(rectifiedL, (0, y), (rectifiedL.shape[1], y), COLORS.RED, 1, lineType=cv2.LINE_AA)
                    for k in range(rectifiedR.shape[0] // interval):
                        y = interval * (k + 1)
                        cv2.line(rectifiedR, (0, y), (rectifiedR.shape[1], y), COLORS.RED, 1, lineType=cv2.LINE_AA)
                        
                    display_frames['rectified L'] = rectifiedL
                    display_frames['rectified R'] = rectifiedR
                    
                    nbeasy_widget_display(display_frames, w_video, resize="auto")
                else:
                    raise NotImplemented
        except (StopIteration, RuntimeError):
            pass
        finally:
            camera.close()
            w_btn.disabled = False

    w_video.width, w_video.height = camera_resolution
    camera = StereoCamera(camera_source[0], camera_resolution, is_3d)
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()
    
    
def on_start_matcher(
    ctx, w_btn, camera_source, camera_resolution, flip, save_dir, rectification_result, w_video,
    num_disparities, uniqueness_ratio, window_size, speckle_size, speckle_range,
    w_enable_visual_depth, w_enable_collect, w_regdata, w_reg_M, w_x, w_y, w_disp):
    
    ctx.logger('on_start_matcher(%s, %s, %d, %d, %d, %d, %d)' % (
        ':'.join(camera_source), save_dir, num_disparities, window_size, speckle_size, speckle_range, uniqueness_ratio), clear=1)
    is_3d = camera_source[1] == 'binocular'
    if not is_3d:
        ctx.logger('not 3d camera')
        return
    
    def _solving_M(value_pairs):
        # depth = M * (1/disparity)
        z = value_pairs[:, 0]
        disp = value_pairs[:, 1]
        disp_inv = 1 / disp
        coeff = np.vstack([disp_inv, np.ones(len(disp_inv))]).T
        ret, sol = cv2.solve(coeff, z, flags=cv2.DECOMP_QR)
        M = np.round(sol[0, 0], 2)
        return M
    
    def _distance_avg(x, y, disp, window_size=3):
        average = []
        m = int(window_size * 0.5)
        for u in range (-1 * m, m + 1):
            for v in range (-1 * m, m + 1):
                average.append(disp[y+u, x+v])
        return sum(average) / len(average)
        
    def _camera_capture(camera, w_video, save_dir):
        try:
            w_btn.disabled = True
            params = json.loads(rectification_result)
            l_map, r_map = np.load(params['l_map_npz']), np.load(params['r_map_npz'])
            l_map_x, l_map_y = l_map['l_map_x'], l_map['l_map_y']
            r_map_x, r_map_y = r_map['r_map_x'], r_map['r_map_y']
            
            Q = np.asarray(params['Q'])
            M = params['M'] if 'M' in params else 0
            if 'M' in params:
                w_reg_M.value = params['M']
            if w_enable_collect.value:
                w_reg_M.value = 0
            matcher = cv2.StereoSGBM_create(
                minDisparity=0,
                numDisparities=16*num_disparities,  # specify the acceptable range for which pixels can move
                blockSize=window_size,              # to normalize brightness and enhance texture
                P1=8 * 3 * (window_size ** 2),      # 惩罚系数 控制视差平滑度，一般：P1=8*通道数*SADWindowSize*SADWindowSize
                P2=32 * 3 * (window_size ** 2),     # P2=4*P1, p2值越大，差异越平滑   
                # disp12MaxDiff=1,                  # the maximum diff between the disparities calculated from left to right and those from right to left.
                # preFilterCap=64,
                uniquenessRatio=uniqueness_ratio,   # post processing: a threshold for the match value to remove false positives
                speckleWindowSize=speckle_size,     # near the boundaries of objects, one picture can see “behind” while the other can’t
                speckleRange=speckle_range,
                mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
            matcherL = matcher
            
            if w_enable_visual_depth.value:
                matcherR = cv2.ximgproc.createRightMatcher(matcherL)
                wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=matcherL)
                wls_filter.setLambda(80000)
                wls_filter.setSigmaColor(1.3)
            
            point_pairs = None
            focal_length = Q[2, 3]
            baseline = 1 / Q[3, 2]
            ctx.logger(f'baseline = {baseline} mm focal_length = {focal_length} pixel')
            BF = round(baseline * focal_length, 2)
            
            minDisp = matcherL.getMinDisparity()
            numDisp = matcherL.getNumDisparities()
            ctx.logger(f'minDisp: {minDisp}, numDisp: {numDisp}, speckleWindowSize: {speckle_size}')
            for idx, frames in camera.read(flip=flip):
                display_images = {}
                if len(frames) == 2:
                    imgL, imgR = frames
                    # 畸变校正和立体校正
                    rectifiedL = cv2.remap(imgL, l_map_x, l_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    rectifiedR = cv2.remap(imgR, r_map_x, r_map_y, cv2.INTER_LINEAR, borderValue=cv2.BORDER_CONSTANT)
                    
                    grayL = cv2.cvtColor(rectifiedL, cv2.COLOR_BGR2GRAY)
                    grayR = cv2.cvtColor(rectifiedR, cv2.COLOR_BGR2GRAY)

                    dispL = matcherL.compute(grayL, grayR)
                    if w_enable_visual_depth.value:
                        dispR = matcherR.compute(grayR, grayL)
                        filter_depth = wls_filter.filter(dispL, imgL, None, dispR)
                        filter_depth = filter_depth.astype(np.float32) / 16.
                        ctx.logger(f'{filter_depth}')
                        filter_depth[filter_depth < minDisp] = minDisp
                        filter_depth[filter_depth > numDisp] = numDisp
                        filter_depth = np.clip(filter_depth, 0, 6666)
                        filter_depth = cv2.normalize(src=filter_depth, dst=filter_depth, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
                        display_images['WFS Depth'] = cv2.applyColorMap(np.uint8(filter_depth), cv2.COLORMAP_JET)
                        
                    dispL = dispL.astype(np.float32) / 16.
                    dispL[dispL < minDisp] = minDisp
                    dispL[dispL > numDisp] = numDisp
                    points_3d = cv2.reprojectImageTo3D(dispL, Q)
                    
                    # 9.26 
                    # disparity[disparity <= 0] = disparity.max()
                    # disparity = (disparity / 16.0 - minDisp) / numDisparities
                    ## assert np.all(disparity > 0)
                    
                    ix, iy = w_x.value, w_y.value
                    cv2.circle(rectifiedL, (ix, iy), radius=speckle_size, color=COLORS.ORANGE, thickness=-1)
                    X, Y, Z = np.int16(points_3d[iy, ix]).tolist()
                    # d = get_depth(disparity, Q)[ix, iy]
                    cv2.putText(
                        rectifiedL, 
                        f'{X}, {Y}, {Z}',
                        (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                    display_images['L'] = rectifiedL
                    
                    nbeasy_widget_display(display_images, w_video, resize="auto", ctx=ctx)
                    
                    # if w_enable_collect.value:
                    #     w_disp.value = round(disparity[iy, ix], 5)
                    #     cv2.putText(
                    #         rectifiedL, 
                    #         f'Disp: {w_disp.value}',
                    #         (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                    # else:
                    #     if w_reg_M.value  == 0 and len(w_regdata.options) > 0:
                    #         point_pairs = np.array([[o[0], o[1]] for _, o in w_regdata.options]).reshape((-1, 1, 2))
                    #         value_pairs = np.array([[o[3], o[2]] for _, o in w_regdata.options])
                    #         w_reg_M.value = _solving_M(value_pairs)
# 
                    #     # if point_pairs is not None:
                    #     #     rectifiedL = cv2.polylines(rectifiedL, [point_pairs], True, COLORS.WHITE, thickness=2)
                    #         
                    # if w_reg_M.value != 0:
                    #     distances = w_reg_M.value / disparity
                    #     D1 = _distance_avg(ix, iy, distances, window_size)
                    #     D2 = distances[iy, ix]
                    #     cv2.putText(
                    #         rectifiedL, 
                    #         f'M:{w_reg_M.value}, BF:{BF} D1:{D1:.2f} D2:{D2:.2f}',
                    #         (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                    #         
                    # nbeasy_widget_display(display_images, w_video, resize="auto")
                    # 
                    # # disparity = matcherL.compute(rectifiedL, rectifiedR).astype(np.float32) / 16.0
                    # # optimal_disparity = (disparity - minDisp) / numDisparities
                    # # distance = baseline * focal_length / optimal_disparity
                    # # distance = distance[int(0.5 * optimal_disparity.shape[0]), int(0.5 * optimal_disparity.shape[1])]
                    # # # distance = -593.97*distance**(3) + 1506.8*distance**(2) - 1373.1*distance + 522.06
                    # # cv2.putText(
                    # #     rectify_image, 
                    # #     f'3D(center): {int(focal_length)} {int(baseline)} {distance}',
                    # #     (int(0.2 * W), int(0.2 * H)), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
# 
                    # # dispL = np.uint16(matcherL.compute(grayL, grayR))
                    # # dispR = np.uint16(matcherR.compute(grayR, grayL))
                    # # dispL = np.uint16(filter_.filter(dispL, grayL, None, dispR))
                    # # dispL[dispL < 0] = 0
                    # # dispL = dispL.astype(np.float32) / 16.
                    # # disparity = (disparity/16.0 - minDisparity)/numDisparities
                    # # 
                    # # points_3d = cv2.reprojectImageTo3D(dispL, Q)
                    # # points_3d = np.round(np.asarray(points_3d, dtype=np.float32), 2)
                    # # 
                    # # distance = points_3d[int(0.5 * dispL.shape[0]), int(0.5 * dispL.shape[1])][2]
                    # # 
                    # # cv2.putText(
                    # #     rectify_image, 
                    # #     f'3D(center): {distance}',
                    # #     (int(0.2 * W), int(0.7 * H)), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS.BLUE, 2)
                    # # nbeasy_widget_display(rectify_image, w_video)
                else:
                    raise NotImplemented
        except (StopIteration, RuntimeError):
            pass
        except Exception:
            ctx.logger(traceback.format_exc(limit=3))
        finally:
            camera.close()
            w_btn.disabled = False

    w_video.width, w_video.height = camera_resolution
    camera = StereoCamera(camera_source[0], camera_resolution, is_3d)
    ctx.logger(f'video shape: {camera.open()}')
    ctx.stereo_camera = camera
    calibrate_thread = threading.Thread(target=_camera_capture, name='stereocamera', args=(camera, w_video, save_dir))
    calibrate_thread.daemon = True
    calibrate_thread.start()

    
on_stop_matcher = on_stop_collect_samples


def on_remove_sample(ctx, w_btn, w_sample_list):
    sample_path = w_sample_list.value
    ctx.logger(f'on_remove_sample({sample_path})')
    if os.path.exists(sample_path):
        os.remove(sample_path)
        if 'binocular' in sample_path:
            os.remove(sample_path.replace('left', 'right'))
    index = w_sample_list.index
    options = list(copy.copy(w_sample_list.options))
    ctx.logger(f'remove: {options.pop(index)}')
    w_sample_list.options = options
    w_sample_list.index = index if index < len(options) else index - 1


def on_refresh_sample(ctx, w_btn, w_sample_list, save_dir):
    ctx.logger(f'on_refresh_sample({save_dir})')
    
    options = []
    if 'binocular' in save_dir:
        left_sample_list = sorted(glob.glob(f'{save_dir}/left_out_*.png'))
        right_sample_list = sorted(glob.glob(f'{save_dir}/right_out_*.png'))
        ctx.logger(f'left samples count: {len(left_sample_list)}, right samples count: {len(right_sample_list)}')
        assert len(left_sample_list) == len(right_sample_list)
        for imgpath in left_sample_list:
            imgfile = os.path.basename(imgpath)
            options.append((imgfile[5:-4], imgpath))
    else:
        sample_list = sorted(glob.glob(f'{save_dir}/out_*.png'))
        for imgpath in sample_list:
            imgfile = os.path.basename(imgpath)
            options.append((imgfile[:-4], imgpath))
    
    w_sample_list.options = options


def on_start_calibration(ctx, w_btn, w_sample_list, w_calibration_result, win_size, term_iters, term_eps, flags):
    try:
        w_btn.disabled = True
        w_calibration_result.value = ''
        options = w_sample_list.options
        ctx.logger(f'on_start_calibration({len(options)}, {flags})', clear=1)
        if len(options) == 0:
            return

        save_dir = os.path.dirname(options[0][1])
        segs = os.path.basename(save_dir).split('_')
        squares_x, squares_y, square_size = list(map(int, segs))
        pattern_size = (squares_x - 1, squares_y - 1) # Number of inner corners per a chessboard row and column
        world_point = np.zeros((np.prod(pattern_size), 3), np.float32)
        world_point[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
        world_point = world_point * square_size

        win_size = (win_size, win_size)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)

        ctx.logger(f'on_start_calibration({win_size}, {pattern_size}, {criteria})')
        image_files = [x[1].replace('out_', 'ori_') for x in options]
        if 'binocular' in save_dir:
            image_L_files = image_files
            image_R_files = [x.replace('left', 'right') for x in image_L_files]
            image_size, error, K1, D1, K2, D2, R, T, E, F = calibrate_binocular_camera(
                ctx, image_L_files, image_R_files, world_point, pattern_size, criteria, win_size, sum(flags))
            result = {
                'image_size': image_size,
                'error': error,
                'K1': K1, 'D1': D1,
                'K2': K2, 'D2': D2,
                'R': R, 'T': T, 'E': E, 'F': F
            }
        else:
            image_size, error, mtx, dist = calibrate_monocular_camera(ctx, image_files, world_point, pattern_size, criteria, win_size)
            result = {
                'image_size': image_size,
                'error': error,
                'K': mtx, 'D': dist
            }
        w_calibration_result.value = json.dumps(result, indent=4)
    finally:
        w_btn.disabled = False


def on_save_calibration_result(ctx, w_btn, calibration_result, save_dir):
    ctx.logger(f'on_save_calibration_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    with open(f'{save_dir}/{prefix}_calibration.json', 'w') as f:
        f.write(calibration_result)
        
    
def on_load_calibration_result(ctx, w_btn, w_calibration_result, save_dir):
    ctx.logger(f'on_load_calibration_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    try:
        file = f'{save_dir}/{prefix}_calibration.json'
        with open(file, 'r') as f:
            w_calibration_result.value = f.read()
    except Exception:
        w_calibration_result.value = f'Open {file} Error'
        

def on_start_rectify(ctx, w_btn, calibration_result, w_rectify_result, save_dir, alpha):
    ctx.logger(f'on_start_rectify({alpha})')
    try:
        w_btn.disabled = True 
        w_rectify_result.value = ''
        params = json.loads(calibration_result)
        K1, D1 = np.asarray(params['K1']), np.asarray(params['D1'])
        K2, D2 = np.asarray(params['K2']), np.asarray(params['D2'])
        image_size, R, T = params['image_size'], np.asarray(params['R']), np.asarray(params['T'])
        l_map_x, l_map_y, r_map_x, r_map_y, R1, R2, P1, P2, Q = rectify_binocular_camera(
            ctx, K1, D1, K2, D2, image_size, R, T, alpha)
        
        np.savez(f'{save_dir}/l_map.npz', l_map_x=l_map_x, l_map_y=l_map_y)
        np.savez(f'{save_dir}/r_map.npz', r_map_x=r_map_x, r_map_y=r_map_y)
        
        result = {
            **params,
            'l_map_npz': f'{save_dir}/l_map.npz',
            'r_map_npz': f'{save_dir}/r_map.npz',
            'R1': R1, 'R2': R2, 'P1': P1, 'P2': P2, 'Q': Q
        }
        # ctx.logger(f'{Q[2, 3]}, {-1 / Q[3, 2]}, {T[0]}')
        w_rectify_result.value = json.dumps(result, indent=4)
    except Exception:
        ctx.logger(f'{traceback.format_exc(limit=6)}')
        ctx.logger(f'{calibration_result}')
    finally:
        w_btn.disabled = False


def on_save_rectification_result(ctx, w_btn, rectification_result, save_dir, reg_M):
    ctx.logger(f'on_save_rectification_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    data = json.loads(rectification_result)
    data['M'] = reg_M
    with open(f'{save_dir}/{prefix}_rectification.json', 'w') as f:
        f.write(json.dumps(data, indent=4))


def on_load_rectification_result(ctx, w_btn, w_rectification_result, save_dir):
    ctx.logger(f'on_load_rectification_result({save_dir})')
    prefix = 'binocular' if 'binocular' in save_dir else 'monocular'
    try:
        file = f'{save_dir}/{prefix}_rectification.json'
        with open(file, 'r') as f:
            w_rectification_result.value = f.read()
    except Exception:
        w_rectification_result.value = f'Open {file} Error'

    
def on_add_regression_data(ctx, w_btn, w_regdata, ix, iy, disp, wd):
    ctx.logger(f'on_save_regression_data({ix}, {iy}, {disp}, {wd})')
    options = list(w_regdata.options)
    if wd == 0:
        return
    options.append([wd, (int(ix), int(iy), disp, wd)])
    w_regdata.options = options
    w_regdata.index = len(w_regdata.options) - 1

#### On Interactive Events

In [8]:
def on_update_sample_dir(
    ctx, w_save_dir, w_calibration_result, w_rectification_result, camera_source, chessboard):
    squares_x, squares_y, square_size = chessboard
    save_dir = f'out/{camera_source[1]}/chessboard/{squares_x}_{squares_y}_{square_size}'
    w_save_dir.value = save_dir
    ctx.logger(f'on_update_sample_dir({save_dir})')
    
    on_load_calibration_result(ctx, None, w_calibration_result, save_dir)
    if camera_source[1] == 'binocular':
        on_load_rectification_result(ctx, None, w_rectification_result, save_dir)


def on_update_sample_list(ctx, w_sample_list, save_dir):
    ctx.logger(f'on_update_sample_list({save_dir})')
    if not os.path.isdir(save_dir):
        return
    on_refresh_sample(ctx, None, w_sample_list, save_dir)


def on_update_sample_image(ctx, w_video_frame, sample_path):
    ctx.logger(f'on_update_sample_image({sample_path})')
    if sample_path is None or not os.path.exists(sample_path):
        ctx.logger(f'not found {sample_path}')
        return
    if 'binocular' in sample_path:
        imgL = cv2.imread(sample_path)
        imgR = cv2.imread(sample_path.replace('left', 'right'))
        nbeasy_widget_display({'L': imgL, 'R': imgR}, w_video_frame, resize='auto')
    else:
        img = cv2.imread(sample_path)
        nbeasy_widget_display(img, w_video_frame)

#### On Canvas Events

In [9]:
def on_canvas_video_mouse_up(ctx, w_canvas, x, y, w_ix, w_iy):
    ctx.logger(f'on_canvas_video_mouse_up({x, y})')
    w_ix.value = int(x)
    w_iy.value = int(y)

### Template Schema

In [10]:
# _IMPORT('./easy_widget.py')

def nbeasy_chessboard_choice_objs(nrow, ncol, square_size, width=300, ro=True):
    return {
        'type': 'H',
        'objs': [
            nbeasy_widget_int('cfg.chessboard_squares_x', 'Squares X', nrow, width=width, readonly=ro),
            nbeasy_widget_int('cfg.chessboard_squares_y', 'Squares Y', ncol, width=width, readonly=ro),
            nbeasy_widget_int('cfg.chessboard_square_size', 'Square Size(mm)', square_size, width=width, readonly=ro)        
        ]
    }

FLAGS = [
    'cv2.CALIB_FIX_INTRINSIC',
    'cv2.CALIB_USE_INTRINSIC_GUESS',
    'cv2.CALIB_FIX_ASPECT_RATIO',
    'cv2.CALIB_ZERO_TANGENT_DIST',
    'cv2.CALIB_SAME_FOCAL_LENGTH',
    'cv2.CALIB_RATIONAL_MODEL',
    'cv2.CALIB_FIX_K3',
    'cv2.CALIB_FIX_K4',
    'cv2.CALIB_FIX_K5'
]

W, H = 640, 480 

schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'Capture',
                    'objs': [
                        {
                            'type': 'V',
                            'name': 'Chessboard',
                            'objs': [
                                nbeasy_widget_stringenumtrigger(
                                    '_cfg.chessboard_choice', 'Board', default=0,
                                    enums = [('A4-30mm-9x7', (9, 7, 30)), ('A3-25mm-16x11', (16, 11, 25))],
                                    triggers = [
                                        nbeasy_chessboard_choice_objs(9, 7, 30),
                                        nbeasy_chessboard_choice_objs(16, 11, 25),
                                    ],
                                    width=333),
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_int('cfg.chessboard_win_size', 'Win Size', default=5, tips='search area'),
                                        nbeasy_widget_int('cfg.chessboard_term_iters', 'Term Iters', default=100, tips='number of iteration'),
                                        nbeasy_widget_float('cfg.chessboard_term_eps', 'Term EPS', default=1e-5, tips='accuracy'),
                                    ]
                                },
                            ]
                        }, # end Chessboard
                        { 'type': 'html', 'text': '<hr>'},
                        {
                            'type': 'V',
                            'name': 'Samples',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_stringenum('cfg.select_camera_source', 'Select Camera', default=-1, enums=_get_cameras()),
                                        nbeasy_widget_int('cfg.sample_size', 'Sample Size', 32),
                                        nbeasy_widget_stringenum(
                                            f'cfg.select_camera_resolution', 'Resolution', default=1,
                                            enums=[('640x480', [640, 480]), ('1280x960', [1280, 960])],
                                        ),
                                        nbeasy_widget_stringenum(
                                            'cfg.select_flip', 'Flip',
                                            enums=[
                                                ('None', -2),
                                                ('Horizontal', 1),
                                                ('Vertical', 0),
                                                ('Both', -1)
                                            ],
                                        ),
                                    ],
                                },
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_string('cfg.sample_save_dir', 'Save Dir', "", width=605, readonly=True),
                                        nbeasy_widget_bool('cfg.rm_out', '<font color="red">Clear Images</font>', False),
                                    ]
                                },
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.btn_start_collect_samples', 'Start', style='success', icon='camera'),
                                                nbeasy_widget_button('__cfg.btn_stop_collect_samples', 'Stop', style='success', icon='stop-circle'),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        nbeasy_widget_image('__cfg.video_frame_capture', 'Frame', '', width=W, height=H),
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_stringenum('cfg.sample_list', 'Sample', width=200, description_width=0, btn_prev=True, btn_next=True),
                                                nbeasy_widget_button('__cfg.btn_del_sample', 'Remove', style='danger', icon='trash', width=100),
                                                nbeasy_widget_button('__cfg.btn_refresh_sample', 'Refresh', style='info', icon='refresh', width=100),
                                            ],
                                            'justify_content': 'center'
                                        }
                                    ],
                                    'align_items': 'center'
                                },
                            ],
                        }, # end Samples
                    ]
                }, # end tab Capture
                {
                    'name': 'Calibrate',
                    'objs': [
                        {
                            'type': 'H',
                            'objs': [
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.start_calibration', 'Calibrate', style='success', icon='check'),
                                                nbeasy_widget_button('__cfg.save_calibration_result', 'Save', style='success', icon='floppy-o'),
                                                nbeasy_widget_button('__cfg.load_calibration_result', 'Load', style='success', icon='spinner'),
                                            ],
                                            'justify_content': 'center'
                                        }, 
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_text('__cfg.calibration_result', '', width=500, height=290)
                                            ],
                                            'justify_content': 'center',
                                            'height': 310,
                                        },
                                    ],
                                    'width': '40%',
                                },
                                {
                                    'type': 'V',
                                    'objs': [
                                        {'type': 'html', 'text': '<center><span><b>Flags</b></span></center>'},
                                        nbeasy_widget_multiselect_simple(
                                            'cfg.stereo_calibrate_flags', '',
                                            default=[0],
                                            enums=[('NONE', 0)] + [(x.split('.')[1], eval(x)) for x in FLAGS],
                                            height=180,
                                        ),
                                        {'type': 'html', 'text': '<center><span><b>Alpha</b></span></center>'},
                                        nbeasy_widget_float('cfg.rectify_alpha', '', default=0, min_=-1, max_=1, step=0.1)
                                    ],
                                    'width': '20%'
                                },
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.start_rectification', 'Rectify', style='success', icon='check'),
                                                nbeasy_widget_button('__cfg.save_rectification_result', 'Save', style='success', icon='floppy-o'),
                                                nbeasy_widget_button('__cfg.load_rectification_result', 'Load', style='success', icon='spinner'),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_text('__cfg.rectification_result', '', width=520, height=290)
                                            ],
                                            'justify_content': 'center',
                                            'height': 310,
                                        },
                                    ],
                                    'width': '40%'
                                },
                            ],
                            'align_items': 'center',
                        }, # end H
                        {
                            'type': 'V',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_button('__cfg.btn_start_c_test',  'Test Calibration', style='success', icon='camera'),
                                        nbeasy_widget_button('__cfg.btn_stop_test', 'Stop Test', style='success', icon='stop-circle'),
                                        nbeasy_widget_button('__cfg.btn_start_r_test', 'Test Rectification', style='success', icon='camera'),
                                    ],
                                    'justify_content': 'center'
                                },
                                nbeasy_widget_image('__cfg.video_frame_test', 'Frame', '', width=W, height=H),
                            ],
                            'align_items': 'center'
                        },
                    ]
                }, # end tab Calibration
                {
                    'name': 'Matcher',
                    'objs': [
                        {
                            'type': 'V',
                            'objs': [
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_int('cfg.num_disp', 'Num Disparities', 16, min_=1, max_=16, width=280),
                                                nbeasy_widget_int('cfg.uniqueness_ratio', 'Uniqueness Ratio', 10, min_=-1, max_=25, width=280),
                                                nbeasy_widget_int('cfg.sad_winsize', 'SAD Window Size', 11, min_=3, max_=27, step=2, slider=True),
                                                nbeasy_widget_int('cfg.speckle_size', 'Speckle Size', 15, min_=10, max_=200, step=5, slider=True),
                                                nbeasy_widget_stringenum('cfg.speckle_range', 'Speckle Rang', 0, enums=[('1', 1), ('2', 2), ('3', 3)]),
                                            ]
                                        },
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_button('__cfg.btn_start_matcher', 'Start', style='success', icon='camera'),
                                                nbeasy_widget_button('__cfg.btn_stop_matcher', 'Stop', style='success', icon='stop-circle'),
                                                nbeasy_widget_bool('cfg.enable_visual_depth', 'Visual Depth', default=False, width=200, description_width=0),
                                                nbeasy_widget_bool('cfg.enable_regression_samples', 'Collect D Samples', default=False, width=200, description_width=0),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        nbeasy_widget_canvas('__cfg.video_frame_matcher', width=W, height=H),
                                    ],
                                    'align_items': 'center'
                                },
                                {
                                    'type': 'V',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_int('cfg.image_x', 'Image X', width=250, readonly=True),
                                                nbeasy_widget_int('cfg.image_y', 'Image Y', width=250, readonly=True),
                                                nbeasy_widget_float('cfg.disp', 'Disp', width=250, readonly=True),
                                                nbeasy_widget_float('cfg.world_d', 'World D', width=250),
                                            ],
                                            'justify_content': 'center'
                                        },
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_stringenum('cfg.regression_data', 'Regress Data', width=250, btn_delete=True),
                                                nbeasy_widget_button('__cfg.btn_reg_data_add', 'Add', width=100, style='success', icon='add'),
                                                nbeasy_widget_float('cfg.reg_M', 'M', default=0, width=250, readonly=False),
                                                nbeasy_widget_button('__cfg.btn_reg_M_save', 'Save M', width=100, style='success', icon='save'),
                                            ],
                                            'justify_content': 'center'
                                        },
                                    ],
                                    'align_items': 'center'
                                },
                            ],
                        },
                    ]
                }, # end tab Matcher
            ]
        }, # end tab
    ], # end pages
    'evts': [
        {
            'type': 'jslink',
            'objs': [
                {
                    'source': '__cfg.btn_start_collect_samples:disabled',
                    'target': '__cfg.btn_start_matcher:disabled'
                },
                {
                    'source': '__cfg.btn_start_collect_samples:disabled',
                    'target': '__cfg.btn_start_c_test:disabled'
                },
                {
                    'source': '__cfg.btn_start_collect_samples:disabled',
                    'target': '__cfg.btn_start_r_test:disabled'
                },
                {
                    'source': '__cfg.btn_start_r_test:disabled',
                    'target': '__cfg.btn_start_c_test:disabled'
                },
            ]
        }, # end jslink
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': on_start_collect_samples,
                    'params': {
                        'sources': ['__cfg.btn_start_collect_samples'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_camera_resolution:value',
                            'cfg.sample_size:value',
                            'cfg.select_flip:value',
                            'cfg.rm_out:value',
                            'cfg.sample_save_dir:value',
                            'cfg.chessboard_win_size:value',
                            'cfg.chessboard_term_iters:value',
                            'cfg.chessboard_term_eps:value',
                            '__cfg.video_frame_capture',
                        ]
                    }
                }, # end event start collect samples
                {
                    'handler': on_stop_collect_samples,
                    'params': {
                        'sources': ['__cfg.btn_stop_collect_samples'],
                        'targets': []
                    }
                }, # end event stop collect samplesn
                {
                    'handler': on_start_calibrition_test,
                    'params': {
                        'sources': ['__cfg.btn_start_c_test'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_camera_resolution:value',
                            'cfg.select_flip:value',
                            'cfg.sample_save_dir:value',
                            '__cfg.calibration_result:value',
                            '__cfg.video_frame_test',
                        ]
                    }
                }, # end event start c test
                {
                    'handler': on_stop_test,
                    'params': {
                        'sources': ['__cfg.btn_stop_test'],
                        'targets': []
                    }
                }, # end event stop c test
                {
                    'handler': on_start_rectification_test,
                    'params': {
                        'sources': ['__cfg.btn_start_r_test'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_camera_resolution:value',
                            'cfg.select_flip:value',
                            'cfg.sample_save_dir:value',
                            '__cfg.rectification_result:value',
                            '__cfg.video_frame_test',
                        ]
                    }
                }, # end event start c test
                {
                    'handler': on_start_matcher,
                    'params': {
                        'sources': ['__cfg.btn_start_matcher'],
                        'targets': [
                            'cfg.select_camera_source:value',
                            'cfg.select_camera_resolution:value',
                            'cfg.select_flip:value',
                            'cfg.sample_save_dir:value',
                            '__cfg.rectification_result:value',
                            '__cfg.video_frame_matcher',
                            'cfg.num_disp:value', 'cfg.uniqueness_ratio:value',
                            'cfg.sad_winsize:value', 'cfg.speckle_size:value', 'cfg.speckle_range:value',
                            'cfg.enable_visual_depth', 'cfg.enable_regression_samples',
                            'cfg.regression_data',
                            'cfg.reg_M',
                            'cfg.image_x',
                            'cfg.image_y',
                            'cfg.disp'
                        ]
                    }
                }, # end event start matcher
                {
                    'handler': on_stop_matcher,
                    'params': {
                        'sources': ['__cfg.btn_stop_matcher'],
                        'targets': []
                    }
                }, # end event stop matcher
                {
                    'handler': on_remove_sample,
                    'params': {
                        'sources': ['__cfg.btn_del_sample'],
                        'targets': [
                            'cfg.sample_list' 
                        ]
                    }
                }, # end event remove sample
                {
                    'handler': on_refresh_sample,
                    'params': {
                        'sources': ['__cfg.btn_refresh_sample'],
                        'targets': [
                            'cfg.sample_list',
                            'cfg.sample_save_dir:value'
                        ] 
                    }
                }, # end event refresh sample
                {
                    'handler': on_start_calibration,
                    'params': {
                        'sources': ['__cfg.start_calibration'],
                        'targets': [
                            'cfg.sample_list',
                            '__cfg.calibration_result',
                            'cfg.chessboard_win_size:value',
                            'cfg.chessboard_term_iters:value',
                            'cfg.chessboard_term_eps:value',
                            'cfg.stereo_calibrate_flags:value',
                        ]
                    }
                }, # end event calibrator
                {
                    'handler': on_save_calibration_result,
                    'params': {
                        'sources': ['__cfg.save_calibration_result'],
                        'targets': [
                            '__cfg.calibration_result:value',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event on save calibration result
                {
                    'handler': on_load_calibration_result,
                    'params': {
                        'sources': ['__cfg.load_calibration_result'],
                        'targets': [
                            '__cfg.calibration_result',
                            'cfg.sample_save_dir:value'
                        ]
                    }
                }, # end event on load calibration result
                {
                    'handler': on_start_rectify,
                    'params': {
                        'sources': ['__cfg.start_rectification'],
                        'targets': [
                            '__cfg.calibration_result:value',
                            '__cfg.rectification_result',
                            'cfg.sample_save_dir:value',
                            'cfg.rectify_alpha:value',
                        ]
                    }
                }, # end event rectify
                {
                    'handler': on_save_rectification_result,
                    'params': {
                        'sources': ['__cfg.save_rectification_result', '__cfg.btn_reg_M_save'],
                        'targets': [
                            '__cfg.rectification_result:value',
                            'cfg.sample_save_dir:value',
                            'cfg.reg_M:value'
                        ]
                    }
                }, # end event on save rectification result
                {
                    'handler': on_load_rectification_result,
                    'params': {
                        'sources': ['__cfg.load_rectification_result'],
                        'targets': [
                            '__cfg.rectification_result',
                            'cfg.sample_save_dir:value',
                        ]
                    }
                }, # end event on load rectification result
                {
                    'handler': on_add_regression_data,
                    'params': {
                        'sources': ['__cfg.btn_reg_data_add'],
                        'targets': [
                            'cfg.regression_data',
                            'cfg.image_x:value',
                            'cfg.image_y:value',
                            'cfg.disp:value',
                            'cfg.world_d:value',
                        ]
                    }
                }, # end event on save regression data
            ]
        }, # end onclick events
        {
            'type': 'interactiveX',
            'objs': [
                {
                    'handler': on_update_sample_dir,
                    'params': {
                        'w_save_dir': 'cfg.sample_save_dir',
                        'w_calibration_result': '__cfg.calibration_result',
                        'w_rectification_result': '__cfg.rectification_result',
                        'camera_source': 'cfg.select_camera_source',
                        'chessboard': '_cfg.chessboard_choice',
                    }
                },
                {
                    'handler': on_update_sample_list,
                    'params': {
                        'w_sample_list': 'cfg.sample_list',
                        'save_dir': 'cfg.sample_save_dir',
                    }
                }, 
                {
                    'handler': on_update_sample_image,
                    'params': {
                        'w_video_frame': '__cfg.video_frame_capture',
                        'sample_path': 'cfg.sample_list'
                    }
                },
            ]
        }, # end interactiveX
        {
            'type': 'oncanvas',
            'objs': [
                {
                    'handler': on_canvas_video_mouse_up,
                    'evttype': 'mouse_up',
                    'params': {
                        'sources': ['__cfg.video_frame_matcher'],
                        'targets': [
                            'cfg.image_x',
                            'cfg.image_y',
                        ]
                    }
                }
                
            ]
        } # end oncanvas
    ] # end events
}

### Startup Test

In [11]:
if g_ctx:
    on_stop_collect_samples(g_ctx)
g_ctx = nbeasy_schema_parse(schema, debug=True, border=False)

Box(children=(Box(children=(VBox(children=(TabE(value=None, children=(VBox(children=(HTML(value="<b><font colo…

In [12]:
raise

RuntimeError: No active exception to reraise

In [ ]:
video = g_ctx.get_widget_byid('__cfg.video_frame_capture')

## References

- [Camera Intrinsic Matrix with Example in Python](https://towardsdatascience.com/camera-intrinsic-matrix-with-example-in-python-d79bf2478c12)
- [What are Intrinsic and Extrinsic Camera Parameters in Computer Vision?](https://towardsdatascience.com/what-are-intrinsic-and-extrinsic-camera-parameters-in-computer-vision-7071b72fb8ec)
- [How to select your Stereo Camera?](https://www.e-consystems.com/blog/camera/products/how-to-select-your-stereo-camera/)
- [What is a stereo vision camera?](https://www.e-consystems.com/blog/camera/technology/what-is-a-stereo-vision-camera-2/) 

- [Tutorial: Stereo 3D reconstruction with openCV using an iPhone camera. Part I.](https://becominghuman.ai/stereo-3d-reconstruction-with-opencv-using-an-iphone-camera-part-i-c013907d1ab5)
- [Tutorial: Stereo 3D reconstruction with OpenCV using an iPhone camera. Part III.](https://medium.com/@omar.ps16/stereo-3d-reconstruction-with-opencv-using-an-iphone-camera-part-iii-95460d3eddf0)

<div class="alert alert-success"><p>
Block matching focuses on high texture images (think a picture of a tree) and semi-global block matching will focus on sub pixel level matching and pictures with more smooth textures (think a picture of a hallway).
</p></div>

![](/notebooks/images/sgbm_num_disparity.png)

- [epipolar geometry & draw it](https://docs.opencv.org/3.4.4/da/de9/tutorial_py_epipolar_geometry.html)

<div class="alert alert-success"><p>
Essential Matrix contains the information about translation and rotation, which describe the location of the second camera relative to the first in global coordinates. </br>
Fundamental Matrix contains the same information as Essential Matrix in addition to the information about the intrinsics of both cameras so that we can relate the two cameras in pixel coordinates. (If we are using rectified images and normalize the point by dividing by the focal lengths, F=E).
</p></div>

- [Stereo Vision: Depth Estimation between object and camera](https://medium.com/analytics-vidhya/distance-estimation-cf2f2fd709d8)

[How Computers See Depth: Recent Advances in Deep Learning-Based Methods](https://towardsdatascience.com/how-computers-see-depth-deep-learning-based-methods-368581b244ed)

![](/notebooks/images/stereo_vision.png)

[3D视觉之立体匹配（Stereo Matching)](https://zhuanlan.zhihu.com/p/161276985)
![](/notebooks/images/disparity_estimation.png)

[JetSon](https://github.com/chawza/jetson-obstacle-avoidance.git)

In [ ]:
    # 计算视差图
    size = (left_image.shape[1], left_image.shape[0])
    if down_scale == False:
        disparity_left = left_matcher.compute(left_image, right_image)
        disparity_right = right_matcher.compute(right_image, left_image)

    else:
        left_image_down = cv2.pyrDown(left_image)
        right_image_down = cv2.pyrDown(right_image)
        factor = left_image.shape[1] / left_image_down.shape[1]

        disparity_left_half = left_matcher.compute(left_image_down, right_image_down)
        disparity_right_half = right_matcher.compute(right_image_down, left_image_down)
        disparity_left = cv2.resize(disparity_left_half, size, interpolation=cv2.INTER_AREA)
        disparity_right = cv2.resize(disparity_right_half, size, interpolation=cv2.INTER_AREA)
        disparity_left = factor * disparity_left
        disparity_right = factor * disparity_right